# Baseline - Linear Interpolation

For a ticker, given a sample period (3-1-2009 to ???), we design a linear interpolation method to to predict its closing price on 3-31-2019:  

predicted_price = price_start + (price_end - price_start) * (end_label - start)/(end - start)  
10_bagger = predicted_price > price_start*10  
 
where:  
* price_start - first valid price in the sample period
* price_end - last valid price in the sample period
* end_label - 3-31-2019
* start - date of the first valid price in the sample period
* end - date of the last valid price in the sample period

This serves as the baseline for other more advanced machine learning methods for 10 Bagger prediction.


In [5]:
import quandl  # Access to Sharadar Core US Equities Bundle
api_key = '7B87ndLPJbCDzpNHosH3'

import math
import platform
import matplotlib
import matplotlib.pyplot as plt
from pylab import rcParams
import numpy as np
import torch
import pandas as pd
from IPython.display import display
import time

from datetime import date, datetime, time, timedelta


print("Python version: ", platform.python_version())
print("Pytorch version: {}".format(torch.__version__))

Python version:  3.6.9
Pytorch version: 1.2.0


## Import Labels

For each sample period (e.g. 3-1-2009 to 12-31-2018), we want to import a list of valid tickers. A valid ticker is defined as a ticker which is active for at least 180 days before the end of the sample period. 

For example, if the end of the sample period is 12-31-2018, a ticker has to be active since 7-4-2018. Any ticker that IPO after 7-4-2018 is not a valid ticker, since there is no enough price history to make an educated prediction.

In [6]:
labels = pd.read_csv("../datasets/sharader/labels_12-31-2018.csv")

y = labels.set_index('ticker')
y['firstpricedate']= pd.to_datetime(y['firstpricedate'])
y['lastpricedate']= pd.to_datetime(y['lastpricedate'])

y.head()

,appreciation,10bagger,table,permaticker,name,exchange,isdelisted,category,cusips,siccode,...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
ticker,,,,,,,,,,,,,,,,,,,,,
A,6.339117,False,SEP,196290,Agilent Technologies Inc,NYSE,N,Domestic,00846U101,3826.0,...,USD,California; U.S.A,2020-01-14,2014-09-26,1999-11-18,2020-01-14,1997-06-30,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.agilent.com
AA,1.224348,False,SEP,124392,Alcoa Corp,NYSE,N,Domestic,013872106,3350.0,...,USD,New York; U.S.A,2020-01-14,2016-11-01,2016-11-01,2020-01-14,2014-12-31,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.alcoa.com
AAAGY,1.275556,False,SEP,120538,Altana Aktiengesellschaft,NYSE,Y,ADR,02143N103,2834.0,...,USD,Jordan,2018-10-16,2018-02-13,2002-05-22,2010-08-12,2000-12-31,2005-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
AAAP,3.331837,False,SEP,155760,Advanced Accelerator Applications SA,NASDAQ,Y,ADR,00790T100,2834.0,...,USD,France,2018-06-28,2016-05-19,2015-11-11,2018-02-09,2012-12-31,2016-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
AAC,0.099459,False,SEP,187592,AAC Holdings Inc,NYSE,Y,Domestic,000307108,8093.0,...,USD,Tennessee; U.S.A,2019-10-25,2015-09-11,2014-10-02,2019-10-25,2013-09-30,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN


### Number of active tickers

In [7]:
tickers = list(y.index)
print(len(tickers))

9881


In [14]:
valid_tickers = pd.Series(tickers, name = 'ticker')

valid_tickers.head()

0        A
1       AA
2    AAAGY
3     AAAP
4      AAC
Name: ticker, dtype: object

In [8]:
prices = pd.read_csv("../datasets/sharader/inputs_notfilled_2018-12-31.csv")
prices.head()

,ticker,"('close', '2009-03-02')","('close', '2009-03-03')","('close', '2009-03-04')","('close', '2009-03-05')","('close', '2009-03-06')","('close', '2009-03-09')","('close', '2009-03-10')","('close', '2009-03-11')","('close', '2009-03-12')",...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
0,A,12.68,12.68,13.31,12.54,12.65,12.44,13.43,13.53,13.81,...,USD,California; U.S.A,2020-01-14,2014-09-26,1999-11-18,2020-01-14,1997-06-30,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.agilent.com
1,AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,USD,New York; U.S.A,2020-01-14,2016-11-01,2016-11-01,2020-01-14,2014-12-31,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.alcoa.com
2,AAAGY,15.75,15.75,16.35,15.59,15.97,15.96,16.20,16.15,16.50,...,USD,Jordan,2018-10-16,2018-02-13,2002-05-22,2010-08-12,2000-12-31,2005-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
3,AAAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,USD,France,2018-06-28,2016-05-19,2015-11-11,2018-02-09,2012-12-31,2016-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
4,AAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,USD,Tennessee; U.S.A,2019-10-25,2015-09-11,2014-10-02,2019-10-25,2013-09-30,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN


In [13]:
X = prices.set_index('ticker')

X = X.filter()

X.head()

,"('close', '2009-03-02')","('close', '2009-03-03')","('close', '2009-03-04')","('close', '2009-03-05')","('close', '2009-03-06')","('close', '2009-03-09')","('close', '2009-03-10')","('close', '2009-03-11')","('close', '2009-03-12')","('close', '2009-03-13')",...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
ticker,,,,,,,,,,,,,,,,,,,,,
A,12.68,12.68,13.31,12.54,12.65,12.44,13.43,13.53,13.81,13.99,...,USD,California; U.S.A,2020-01-14,2014-09-26,1999-11-18,2020-01-14,1997-06-30,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.agilent.com
AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,USD,New York; U.S.A,2020-01-14,2016-11-01,2016-11-01,2020-01-14,2014-12-31,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.alcoa.com
AAAGY,15.75,15.75,16.35,15.59,15.97,15.96,16.20,16.15,16.50,16.36,...,USD,Jordan,2018-10-16,2018-02-13,2002-05-22,2010-08-12,2000-12-31,2005-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
AAAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,USD,France,2018-06-28,2016-05-19,2015-11-11,2018-02-09,2012-12-31,2016-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
AAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,USD,Tennessee; U.S.A,2019-10-25,2015-09-11,2014-10-02,2019-10-25,2013-09-30,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN


In [28]:
start_price_hist = '2009-03-01'
end_price_hist = '2019-03-31'

X = prices.filter(items=['ticker', 'date', 'close'])
X = X[(prices.date >= start_price_hist) & (end_price_hist <= end_date)]

# Pivot the dataframe
X = X.pivot(index ='date', columns ='ticker') 

In [29]:
X.head()

close                                                    ...      \
ticker          A  AA  AAAGY AAAP AAC  AACC  AACG AACPF AAGIY   AAI  ... ZUO   
date                                                                 ...       
2009-03-02  12.68 NaN  15.75  NaN NaN  3.29  5.18   NaN   NaN  2.54  ... NaN   
2009-03-03  12.68 NaN  15.75  NaN NaN  3.30  5.32   NaN   NaN  2.46  ... NaN   
2009-03-04  13.31 NaN  16.35  NaN NaN  3.33  5.08   NaN   NaN  2.78  ... NaN   
2009-03-05  12.54 NaN  15.59  NaN NaN  3.30  5.08   NaN   NaN  2.56  ... NaN   
2009-03-06  12.65 NaN  15.97  NaN NaN  3.40  5.25   NaN   NaN  2.89  ... NaN   

                                                                
ticker      ZURVY ZVO  ZVUE ZXAIY ZYME ZYNE   ZYTO  ZYXI    ZZ  
date                                                            
2009-03-02  12.75 NaN  0.01   NaN  NaN  NaN  0.011  1.21  0.84  
2009-03-03  12.85 NaN  0.01   NaN  NaN  NaN  0.011  1.22  0.76  
2009-03-04  13.74 NaN  0.01   NaN  NaN  NaN  0.011  1.22  0.76  
2009-03-05  11.91 NaN  0.01   NaN  NaN  NaN  0.011  1.17  0.58  
2009-03-06  11.30 NaN  0.01   NaN  NaN  NaN  0.011  1.20  0.55  

[5 rows x 10611 columns]

In [34]:
print (X[('close','A')])

date
2009-03-02    12.68
2009-03-03    12.68
2009-03-04    13.31
2009-03-05    12.54
2009-03-06    12.65
              ...  
2019-03-25    79.52
2019-03-26    80.65
2019-03-27    79.00
2019-03-28    79.70
2019-03-29    80.38
Name: (close, A), Length: 2538, dtype: float64


In [90]:
start_date = '2009-03-01'
end_date = '2019-03-31'

predictions = []

for ticker in tickers:
    
    print("Ticker: {}".format(ticker))
    
    s = X[('close',ticker)]
    
    firstTradingDate = y['firstpricedate'].loc[ticker]
    price_start = X[('close',ticker)].loc[s.first_valid_index()]
    
    lastTradingDate = y['lastpricedate'].loc[ticker]
    price_end = X[('close',ticker)].loc[ s.last_valid_index()]
    
    print("First Trading Date: {}  Price: {}".format(firstTradingDate,price_start))
    print("Last Trading Date: {}  Price: {}".format(lastTradingDate, price_end ))
    
    delta = (lastTradingDate - firstTradingDate).days
    
    # For now, set the two to be the same
    predict_last_price = price_end
    
    appreciation = predict_last_price/price_start
    predictions.append(appreciation > 10.0)
    
    print(appreciation, appreciation > 10.0)

Ticker: A
First Trading Date: 1999-11-18 00:00:00  Price: 12.68
Last Trading Date: 2020-01-14 00:00:00  Price: 80.38
6.339116719242902 False
Ticker: AA
First Trading Date: 2016-11-01 00:00:00  Price: 23.0
Last Trading Date: 2020-01-14 00:00:00  Price: 28.16
1.2243478260869565 False
Ticker: AAAGY
First Trading Date: 2002-05-22 00:00:00  Price: 15.75
Last Trading Date: 2010-08-12 00:00:00  Price: 20.09
1.2755555555555556 False
Ticker: AAAP
First Trading Date: 2015-11-11 00:00:00  Price: 24.5
Last Trading Date: 2018-02-09 00:00:00  Price: 81.63
3.3318367346938773 False
Ticker: AAC
First Trading Date: 2014-10-02 00:00:00  Price: 18.5
Last Trading Date: 2019-10-25 00:00:00  Price: 1.84
0.09945945945945947 False
Ticker: AACC
First Trading Date: 2004-02-05 00:00:00  Price: 3.29
Last Trading Date: 2013-06-13 00:00:00  Price: 6.51
1.978723404255319 False
Ticker: AACG
First Trading Date: 2008-01-29 00:00:00  Price: 5.18
Last Trading Date: 2020-01-14 00:00:00  Price: 0.992
0.19150579150579153 Fal

First Trading Date: 2014-09-22 00:00:00  Price: 24.49
Last Trading Date: 2020-01-14 00:00:00  Price: 25.82
1.0543078807676602 False
Ticker: AGO-PF
First Trading Date: 2014-09-22 00:00:00  Price: 24.066
Last Trading Date: 2020-01-14 00:00:00  Price: 25.62
1.0645724258289704 False
Ticker: AGP
First Trading Date: 2001-11-06 00:00:00  Price: 23.7
Last Trading Date: 2012-12-21 00:00:00  Price: 91.7
3.869198312236287 False
Ticker: AGR
First Trading Date: 2015-12-17 00:00:00  Price: 36.12
Last Trading Date: 2020-01-14 00:00:00  Price: 50.35
1.3939645625692139 False
Ticker: AGRO
First Trading Date: 2011-01-28 00:00:00  Price: 12.0
Last Trading Date: 2020-01-14 00:00:00  Price: 6.89
0.5741666666666666 False
Ticker: AGRPY
First Trading Date: 2018-05-24 00:00:00  Price: 25.56
Last Trading Date: 2020-01-14 00:00:00  Price: 20.68
0.809076682316119 False
Ticker: AGRX
First Trading Date: 2014-05-23 00:00:00  Price: 5.54
Last Trading Date: 2020-01-14 00:00:00  Price: 1.51
0.27256317689530685 False
Tic

First Trading Date: 2014-09-22 00:00:00  Price: 24.61
Last Trading Date: 2016-08-31 00:00:00  Price: 24.84
1.0093457943925235 False
Ticker: AMTX
First Trading Date: 2014-05-16 00:00:00  Price: 5.825
Last Trading Date: 2020-01-14 00:00:00  Price: 0.8320000000000001
0.1428326180257511 False
Ticker: AMTY
First Trading Date: 1996-10-17 00:00:00  Price: 0.27
Last Trading Date: 2020-01-14 00:00:00  Price: 0.02
0.07407407407407407 False
Ticker: AMWD
First Trading Date: 1986-07-21 00:00:00  Price: 14.5
Last Trading Date: 2020-01-14 00:00:00  Price: 82.63
5.698620689655172 False
Ticker: AMX
First Trading Date: 2001-02-12 00:00:00  Price: 11.83
Last Trading Date: 2020-01-14 00:00:00  Price: 14.28
1.2071005917159763 False
Ticker: AMYRZ
First Trading Date: 2002-08-02 00:00:00  Price: 2.5
Last Trading Date: 2009-11-24 00:00:00  Price: 3.5
1.4 False
Ticker: AMZG
First Trading Date: 2004-11-16 00:00:00  Price: 0.54
Last Trading Date: 2016-12-05 00:00:00  Price: 0.008
0.014814814814814814 False
Ticker

First Trading Date: 1990-03-27 00:00:00  Price: 1.77
Last Trading Date: 2020-01-14 00:00:00  Price: 4.02
2.271186440677966 False
Ticker: AST
First Trading Date: 2014-07-21 00:00:00  Price: 3.6
Last Trading Date: 2019-03-07 00:00:00  Price: 0.945
0.26249999999999996 False
Ticker: ASTC
First Trading Date: 1995-12-21 00:00:00  Price: 1.65
Last Trading Date: 2020-01-14 00:00:00  Price: 3.64
2.206060606060606 False
Ticker: ASTE
First Trading Date: 1986-06-19 00:00:00  Price: 20.33
Last Trading Date: 2020-01-14 00:00:00  Price: 37.76
1.8573536645351698 False
Ticker: ASTI
First Trading Date: 2006-07-25 00:00:00  Price: 24.0
Last Trading Date: 2016-02-24 00:00:00  Price: 0.06
0.0025 False
Ticker: ASTTY
First Trading Date: 2000-07-11 00:00:00  Price: 0.05
Last Trading Date: 2010-02-26 00:00:00  Price: 0.01
0.19999999999999998 False
Ticker: ASTX
First Trading Date: 1996-03-13 00:00:00  Price: 1.55
Last Trading Date: 2013-10-09 00:00:00  Price: 8.49
5.47741935483871 False
Ticker: ASUR
First Tradi

First Trading Date: 2007-03-15 00:00:00  Price: 5.73
Last Trading Date: 2011-11-18 00:00:00  Price: 2.24
0.39092495636998253 False
Ticker: BBNK
First Trading Date: 2003-01-27 00:00:00  Price: 4.25
Last Trading Date: 2015-06-30 00:00:00  Price: 29.8
7.011764705882353 False
Ticker: BBOX
First Trading Date: 1992-12-17 00:00:00  Price: 18.81
Last Trading Date: 2019-01-07 00:00:00  Price: 1.09
0.05794790005316322 False
Ticker: BBQ
First Trading Date: 1996-11-04 00:00:00  Price: 2.45
Last Trading Date: 2020-01-14 00:00:00  Price: 5.68
2.3183673469387753 False
Ticker: BBRG
First Trading Date: 2010-10-21 00:00:00  Price: 15.95
Last Trading Date: 2018-05-23 00:00:00  Price: 4.05
0.25391849529780564 False
Ticker: BBRYF
First Trading Date: 2010-04-05 00:00:00  Price: 11.0
Last Trading Date: 2020-01-13 00:00:00  Price: 24.09
2.19 False
Ticker: BBSI
First Trading Date: 1993-06-14 00:00:00  Price: 8.24
Last Trading Date: 2020-01-14 00:00:00  Price: 77.33
9.384708737864077 False
Ticker: BBT-PD
First 

...

First Trading Date: 2007-02-20 00:00:00  Price: 5.0
Last Trading Date: 2018-12-07 00:00:00  Price: 0.0001
2e-05 False
Ticker: TORC
First Trading Date: 2018-01-26 00:00:00  Price: 16.88
Last Trading Date: 2020-01-14 00:00:00  Price: 6.82
0.4040284360189574 False
Ticker: TORM
First Trading Date: 1990-03-27 00:00:00  Price: 2.7
Last Trading Date: 2020-01-13 00:00:00  Price: 2.55
0.9444444444444443 False
Ticker: TOSYY
First Trading Date: 2010-01-05 00:00:00  Price: 28.291999999999998
Last Trading Date: 2020-01-14 00:00:00  Price: 15.86
0.5605824968188887 False
Ticker: TOT
First Trading Date: 1991-10-25 00:00:00  Price: 43.72
Last Trading Date: 2020-01-14 00:00:00  Price: 55.65
1.2728728270814273 False
Ticker: TOTA
First Trading Date: 2018-08-02 00:00:00  Price: 10.13
Last Trading Date: 2020-01-14 00:00:00  Price: 10.09
0.9960513326752221 False
Ticker: TOUR
First Trading Date: 2014-05-09 00:00:00  Price: 10.07
Last Trading Date: 2020-01-14 00:00:00  Price: 4.8
0.47666335650446867 False
Tick

First Trading Date: 1995-08-18 00:00:00  Price: 8.45
Last Trading Date: 2020-01-14 00:00:00  Price: 10.85
1.2840236686390534 False
Ticker: UBET
First Trading Date: 1999-06-15 00:00:00  Price: 1.25
Last Trading Date: 2010-06-02 00:00:00  Price: 2.48
1.984 False
Ticker: UBFO
First Trading Date: 1999-03-03 00:00:00  Price: 3.635
Last Trading Date: 2020-01-14 00:00:00  Price: 10.6
2.9160935350756536 False
Ticker: UBIQ
First Trading Date: 2013-02-26 00:00:00  Price: 7.0
Last Trading Date: 2017-07-11 00:00:00  Price: 0.05
0.0071428571428571435 False
Ticker: UBNK
First Trading Date: 2005-08-17 00:00:00  Price: 5.331
Last Trading Date: 2019-10-31 00:00:00  Price: 14.35
2.6918026636653534 False
Ticker: UBNK1
First Trading Date: 2005-07-13 00:00:00  Price: 12.39
Last Trading Date: 2014-04-30 00:00:00  Price: 17.64
1.423728813559322 False
Ticker: UBOH
First Trading Date: 2000-03-22 00:00:00  Price: 9.05
Last Trading Date: 2020-01-14 00:00:00  Price: 23.11
2.553591160220994 False
Ticker: UBP
First

Ticker: VMED
First Trading Date: 2004-07-13 00:00:00  Price: 4.65
Last Trading Date: 2013-06-07 00:00:00  Price: 51.0
10.96774193548387 True
Ticker: VMEMQ
First Trading Date: 2013-09-27 00:00:00  Price: 28.08
Last Trading Date: 2017-04-19 00:00:00  Price: 0.011000000000000001
0.0003917378917378918 False
Ticker: VMI
First Trading Date: 1990-03-26 00:00:00  Price: 40.21
Last Trading Date: 2020-01-14 00:00:00  Price: 130.1
3.2355135538423276 False
Ticker: VMW
First Trading Date: 2007-08-15 00:00:00  Price: 20.73
Last Trading Date: 2020-01-14 00:00:00  Price: 180.51
8.70767004341534 False
Ticker: VNBCQ
First Trading Date: 1994-05-04 00:00:00  Price: 0.12
Last Trading Date: 2010-08-02 00:00:00  Price: 0.004
0.03333333333333333 False
Ticker: VNCE
First Trading Date: 2013-11-22 00:00:00  Price: 286.6
Last Trading Date: 2020-01-14 00:00:00  Price: 12.4
0.04326587578506629 False
Ticker: VNDA
First Trading Date: 2006-04-12 00:00:00  Price: 0.8
Last Trading Date: 2020-01-14 00:00:00  Price: 18.4


First Trading Date: 2010-07-14 00:00:00  Price: 172.405
Last Trading Date: 2020-01-13 00:00:00  Price: 0.045
0.0002610133116788956 False
Ticker: WMS
First Trading Date: 2014-07-25 00:00:00  Price: 16.15
Last Trading Date: 2020-01-14 00:00:00  Price: 25.77
1.5956656346749227 False
Ticker: WMS1
First Trading Date: 1986-01-01 00:00:00  Price: 17.44
Last Trading Date: 2013-10-16 00:00:00  Price: 25.98
1.4896788990825687 False
Ticker: WMT
First Trading Date: 1986-01-01 00:00:00  Price: 48.04
Last Trading Date: 2020-01-14 00:00:00  Price: 97.53
2.0301831806827644 False
Ticker: WMTM
First Trading Date: 2004-05-11 00:00:00  Price: 0.4
Last Trading Date: 2016-11-18 00:00:00  Price: 0.04
0.09999999999999999 False
Ticker: WMZ
First Trading Date: 2008-01-18 00:00:00  Price: 15.09
Last Trading Date: 2010-08-31 00:00:00  Price: 31.86
2.1113320079522864 False
Ticker: WNC
First Trading Date: 1991-11-08 00:00:00  Price: 1.57
Last Trading Date: 2020-01-14 00:00:00  Price: 13.55
8.630573248407643 False
T

First Trading Date: 1989-11-10 00:00:00  Price: 1.03
Last Trading Date: 2020-01-14 00:00:00  Price: 6.88
6.679611650485437 False
Ticker: ZKIN
First Trading Date: 2017-09-01 00:00:00  Price: 8.45
Last Trading Date: 2020-01-14 00:00:00  Price: 1.55
0.18343195266272191 False
Ticker: ZLAB
First Trading Date: 2017-09-20 00:00:00  Price: 27.93
Last Trading Date: 2020-01-14 00:00:00  Price: 29.51
1.0565699964196205 False
Ticker: ZLC
First Trading Date: 1993-08-09 00:00:00  Price: 1.12
Last Trading Date: 2014-05-28 00:00:00  Price: 21.16
18.892857142857142 True
Ticker: ZLPSF
First Trading Date: 2018-08-23 00:00:00  Price: 174.0
Last Trading Date: 2020-01-13 00:00:00  Price: 114.0
0.6551724137931034 False
Ticker: ZLTQ
First Trading Date: 2011-10-19 00:00:00  Price: 15.5
Last Trading Date: 2017-04-27 00:00:00  Price: 56.48
3.643870967741935 False
Ticker: ZMTP
First Trading Date: 2009-10-07 00:00:00  Price: 1.6
Last Trading Date: 2020-01-14 00:00:00  Price: 1.0
0.625 False
Ticker: ZN
First Tradin

In [88]:
print(delta)

2533


In [62]:
X[('close','A')].loc[('2009-03-02')]

12.68